In [41]:
# Remove any leading/trailing whitespace from column names

In [86]:
import pandas as pd
import random
import faker

# Initialize the Faker library
fake = faker.Faker()

# Define the number of students
num_students = 2000

# Define years and departments
year_mapping = {
    1: (22, '3'),
    2: (21, '4'),
    3: (23, '2'),
    4: (24, '1')
}
department_codes = {
    'Business Systems': '34',
    'EEE': '02',
    'AIML': '33',
    'CSE': '05',
    'DS': '32',
    'Mech': '07',
    'Civil': '01'
}
data = []
for _ in range(num_students):
    year, year_str = random.choice(list(year_mapping.values()))
    department = random.choice(list(department_codes.keys()))
    department_code = department_codes[department]
    student_id = f"{year}091A{department_code}{random.randint(1, 99)}"
    
    student_data = {
        'ID Number': student_id,
        'Student Name': fake.name(),
        'Department': department,  # Store the department name
        'Year': year_str,
        'Percentage': round(random.uniform(0, 100), 2),
        'Local Hackathons Participated': random.randint(0, 30),
        'Local Hackathons Won': random.randint(0, 10),
        'Paper Presentations': random.randint(0, 10),
        'National Level Hackathons participated': random.randint(0, 5),
        'National Level Hackathons Won': random.randint(0, 5),
        'Certifications': random.randint(0, 30),
        'R&D Work Rating': random.randint(1, 5),
        'Social Interactions': random.randint(0, 10),
        'Paper Publications': random.randint(0, 30),
    }
    
    data.append(student_data)

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('students.csv', index=False)

print("Dataset created successfully with 2000 students.")

Dataset created successfully with 2000 students.


In [126]:
import pandas as pd
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

 


    

In [108]:
df = pd.read_csv('students.csv')
df.isnull().sum()
df.columns = df.columns.str.strip()
columns_to_normalize = [
    'Percentage',
    'Local Hackathons Participated',
    'Local Hackathons Won',
    'Paper Presentations',
    'National Level Hackathons participated',
    'National Level Hackathons Won',
    'Certifications',
    'R&D Work Rating',
    'Social Interactions',
    'Paper Publications'
]

In [110]:
for column in columns_to_normalize:
    if column in df.columns:
        max_value = df[column].max()
        if max_value > 0:  # Prevent division by zero
            df[column] = df[column] / max_value
        else:
            print(f"Max value for '{column}' is zero, skipping normalization.")
    else:
        print(f"Column '{column}' not found in the DataFrame.")
label_encoder = LabelEncoder()
if 'Student Name' in df.columns:
    df['Student Name'] = label_encoder.fit_transform(df['Student Name'])

In [112]:
weights = {
    'Percentage': 0.25,  # 25% weight  # 8% weight
    'Local Hackathons Participated': 0.07,
    'Local Hackathons Won': 0.08,# 7% weight
    'Paper Presentations': 0.08,  # 8% weight
    'Paper Presentations': 0.07,  # 7% weight
    'National Level Hackathons Won': 0.12,  # 12% weight
    'Certifications': 0.1,  # 10% weight
    'R&D Work Rating': 0.08,  # 8% weight
    'Social Interactions': 0.05,  # 5% weight
    'Paper Publications': 0.1   # 10% weight
}

In [114]:
df['Initial Performance Score'] = sum(df[column] * weight for column, weight in weights.items())
df['Rank'] = df['Initial Performance Score'].rank(ascending=False)


In [116]:
top_students_initial = df.nlargest(3, 'Initial Performance Score')
features = df.drop(columns=['ID Number', 'Year', 'Initial Performance Score', 'Rank','Department'])
target = df['Initial Performance Score']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [118]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
model_path = r'C:\Users\kalpa\Desktop\Hackathon\model.pkl'
with open(model_path, 'wb') as file:
    pickle.dump(model, file)

print("Model saved as model.pkl")

Model saved as model.pkl


In [120]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"\nModel Evaluation:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")


Model Evaluation:
Mean Squared Error: 0.0014
R² Score: 0.8820


In [122]:
year_input = int(input("Enter the year to get the top 3 students using ML predictions: "))

# Filter the data for the specified year
year_data = df[df['Year'] == year_input]
if year_data.empty:
    print(f"No data found for the year {year_input}.")
else:
    # Make predictions on the filtered year data
    year_data['Predicted Performance Score'] = model.predict(year_data.drop(columns=['ID Number', 'Year', 'Initial Performance Score', 'Rank','Department']))

    # Getting top 3 students based on predicted performance score
    top_students_ml = year_data[['ID Number', 'Student Name', 'Predicted Performance Score','Department']].nlargest(3, 'Predicted Performance Score')
    
    # Display the top 3 students
    print(f"\nTop 3 Students for the year {year_input} using ML predictions:")
    print(top_students_ml)
    

Enter the year to get the top 3 students using ML predictions:  1



Top 3 Students for the year 1 using ML predictions:
       ID Number  Student Name  Predicted Performance Score Department
1037  24091A3363          1859                     0.695229       AIML
392   24091A3278           384                     0.691967         DS
1318  24091A0555           905                     0.682670        CSE


C:\Users\kalpa\AppData\Local\Temp\ipykernel_54700\853799521.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_data['Predicted Performance Score'] = model.predict(year_data.drop(columns=['ID Number', 'Year', 'Initial Performance Score', 'Rank','Department']))


In [ ]:
#final code
import pandas as pd
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
# Load the dataset
df = pd.read_csv('students.csv')
df.isnull().sum()
df.columns = df.columns.str.strip()
columns_to_normalize = [
    'Percentage',
    'Local Hackathons Participated',
    'Local Hackathons Won',
    'Paper Presentations',
    'National Level Hackathons participated',
    'National Level Hackathons Won',
    'Certifications',
    'R&D Work Rating',
    'Social Interactions',
    'Paper Publications'
]
for column in columns_to_normalize:
    if column in df.columns:
        max_value = df[column].max()
        if max_value > 0:  # Prevent division by zero
            df[column] = df[column] / max_value
        else:
            print(f"Max value for '{column}' is zero, skipping normalization.")
    else:
        print(f"Column '{column}' not found in the DataFrame.")
label_encoder = LabelEncoder()
if 'Student Name' in df.columns:
    df['Student Name'] = label_encoder.fit_transform(df['Student Name'])
weights = {
    'Percentage': 0.25,  # 25% weight  # 8% weight
    'Local Hackathons Participated': 0.07,
    'Local Hackathons Won': 0.08,# 7% weight
    'Paper Presentations': 0.08,  # 8% weight
    'Paper Presentations': 0.07,  # 7% weight
    'National Level Hackathons Won': 0.12,  # 12% weight
    'Certifications': 0.1,  # 10% weight
    'R&D Work Rating': 0.08,  # 8% weight
    'Social Interactions': 0.05,  # 5% weight
    'Paper Publications': 0.1   # 10% weight
}
df['Initial Performance Score'] = sum(df[column] * weight for column, weight in weights.items())
df['Rank'] = df['Initial Performance Score'].rank(ascending=False)

# Display the top 3 students based on the initial score
top_students_initial = df.nlargest(3, 'Initial Performance Score')
features = df.drop(columns=['ID Number', 'Year', 'Initial Performance Score', 'Rank','Department'])
target = df['Initial Performance Score']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train a Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
model_path = r'C:\Users\kalpa\Desktop\Hackathon\model.pkl'
with open(model_path, 'wb') as file:
    pickle.dump(model, file)
with open(model_path, 'wb') as file:
    pickle.dump(model, file)

print("Model saved as model.pkl")
# Make predictions on the test set
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"\nModel Evaluation:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")
year_input = int(input("Enter the year to get the top 3 students using ML predictions: "))

# Filter the data for the specified year
year_data = df[df['Year'] == year_input]
if year_data.empty:
    print(f"No data found for the year {year_input}.")
else:
    # Make predictions on the filtered year data
    year_data['Predicted Performance Score'] = model.predict(year_data.drop(columns=['ID Number', 'Year', 'Initial Performance Score', 'Rank','Department']))

    # Getting top 3 students based on predicted performance score
    top_students_ml = year_data[['ID Number', 'Student Name', 'Predicted Performance Score','Department']].nlargest(3, 'Predicted Performance Score')
    
    # Display the top 3 students
    print(f"\nTop 3 Students for the year {year_input} using ML predictions:")
    print(top_students_ml)